In [12]:
import gc
gc.collect()

0

In [13]:
from google.colab import drive
drive.mount('/content/drive')
import re
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LambdaCallback, ModelCheckpoint
import os
import glob
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.optimizers import Adam
!pip install unidecode
from unidecode import unidecode

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
with open("/content/drive/MyDrive/EV3/Don Quijote de la Mancha.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [15]:
print(f"Longitud total del texto: {len(text)} caracteres\n")
print("Primeros 1000 caracteres:\n")
print(text[:1000])

Longitud total del texto: 2071198 caracteres

Primeros 1000 caracteres:

Capítulo primero. Que trata de la condición y ejercicio del famoso hidalgo
don Quijote de la Mancha


En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor. Una olla de algo más vaca que carnero,
salpicón las más noches, duelos y quebrantos los sábados, lantejas los
viernes, algún palomino de añadidura los domingos, consumían las tres
partes de su hacienda. El resto della concluían sayo de velarte, calzas de
velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de
entresemana se honraba con su vellorí de lo más fino. Tenía en su casa una
ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte,
y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la
podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años; era de
complexión recia, seco de

##Limpieza y Normalizacion

- Convierte todo el texto a minúsculas.
- Elimina acentos y caracteres especiales.
- Quita símbolos no deseados, dejando solo letras, puntuación básica y espacios.
- Reemplaza múltiples espacios por uno solo y elimina espacios al inicio y final.


In [16]:
text = unidecode(text.lower())
text = re.sub(r'[^a-z¿¡.,;:!? ]', ' ', text)
text = re.sub(r'\s+', ' ', text).strip()

print("Texto limpio:")
print(text[:1000])


Texto limpio:
capitulo primero. que trata de la condicion y ejercicio del famoso hidalgo don quijote de la mancha en un lugar de la mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivia un hidalgo de los de lanza en astillero, adarga antigua, rocin flaco y galgo corredor. una olla de algo mas vaca que carnero, salpicon las mas noches, duelos y quebrantos los sabados, lantejas los viernes, algun palomino de anadidura los domingos, consumian las tres partes de su hacienda. el resto della concluian sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los dias de entresemana se honraba con su vellori de lo mas fino. tenia en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que asi ensillaba el rocin como tomaba la podadera. frisaba la edad de nuestro hidalgo con los cincuenta anos; era de complexion recia, seco de carnes, enjuto de rostro, gran madrugador y amigo de la caza

# RNN Palabra y Texto

## Tokenizar

- Crea un tokenizador con un token especial `<OOV>` para palabras fuera del vocabulario.
- Ajusta el tokenizador al texto completo para construir el índice de palabras.
- Calcula la cantidad total de palabras únicas.
- Evita cargar todas las secuencias en memoria desde el inicio, preparándose para procesar por lotes.


In [17]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Se procesará el texto en lotes para evitar problemas de memoria
# No se crea token_list completa en memoria
print("Total de palabras únicas:", total_words)



Total de palabras únicas: 22136


## Secuencias y Padding de entrenamiento

Se define un generador de datos que procesa el texto dividido en oraciones, tokeniza cada una en n-gramas, y genera secuencias de entrada (`X`) y su palabra objetivo (`y`) de forma incremental. Cada secuencia se trunca a una longitud máxima (`MAX_SEQUENCE_LEN = 50`) y se rellena con ceros por la izquierda. La palabra objetivo se convierte a one-hot (`to_categorical`) y los datos se agrupan en lotes (`batch_size`) para optimizar el uso de memoria. Finalmente, se calcula el número total de secuencias posibles y los pasos por época para el entrenamiento del modelo.

In [21]:
MAX_SEQUENCE_LEN = 50
sentences = text.split(".")

def data_generator(sentences, tokenizer, max_sequence_len, total_words, batch_size=32):
    while True:
        batch_X = []
        batch_y = []
        for sentence in sentences:
            token_list = tokenizer.texts_to_sequences([sentence])[0]
            for i in range(1, len(token_list)):
                n_gram_sequence = token_list[:i+1]
                if len(n_gram_sequence) > max_sequence_len: # Truncar secuencias muy largas
                    n_gram_sequence = n_gram_sequence[-max_sequence_len:]

                padded_sequence = pad_sequences([n_gram_sequence], maxlen=max_sequence_len, padding='pre')[0]

                X_seq = padded_sequence[:-1]
                y_word = padded_sequence[-1]

                if y_word == 0:
                    continue

                batch_X.append(X_seq)
                batch_y.append(to_categorical(y_word, num_classes=total_words))

                if len(batch_X) == batch_size:
                    yield np.array(batch_X), np.array(batch_y)
                    batch_X = []
                    batch_y = []

        if len(batch_X) > 0:
            yield np.array(batch_X), np.array(batch_y)


num_sequences = 0
for sentence in sentences:
    token_list = tokenizer.texts_to_sequences([sentence])[0]
    num_sequences += max(0, len(token_list) - 1)

BATCH_SIZE = 128
STEPS_PER_EPOCH = num_sequences // BATCH_SIZE

print(f"MAX_SEQUENCE_LEN: {MAX_SEQUENCE_LEN}")
print(f"BATCH_SIZE: {BATCH_SIZE}")
print(f"STEPS_PER_EPOCH: {STEPS_PER_EPOCH}")


MAX_SEQUENCE_LEN: 50
BATCH_SIZE: 128
STEPS_PER_EPOCH: 2879


## Modelo LSTM y entrenamiento

In [22]:

# === Definición del Modelo ===
model = Sequential([
    Embedding(input_dim=total_words, output_dim=64, input_length=MAX_SEQUENCE_LEN - 1),
    LSTM(128),
    Dense(total_words, activation='softmax')
])

# Construcción explícita del modelo
model.build(input_shape=(None, MAX_SEQUENCE_LEN - 1))

# Compilación
model.compile(
    loss='categorical_crossentropy',  # porque y es one-hot
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Mostrar resumen
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 49, 64)         │     1,416,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 22136)          │     2,855,544 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,371,064 (16.67 MB)

 Trainable params: 4,371,064 (16.67 MB)

 Non-trainable params: 0 (0.00 B)

##Callbacks y Entrenamiento

Se definen tres callbacks principales: `EarlyStopping`, que detiene el entrenamiento si la pérdida de validación no mejora después de 15 épocas; `ReduceLROnPlateau`, que reduce el `learning rate` si la validación se estanca, con un mínimo de `0.00005`; y `ModelCheckpoint`, que guarda el mejor modelo basado en `val_accuracy`. Además, se implementa una clase personalizada `TextGenerator`, que genera texto cada 5 épocas usando el modelo actual y una semilla (`don quijote de la mancha`) para observar el progreso del aprendizaje. Finalmente, el modelo se entrena usando un `data_generator` que produce secuencias por lotes, con 100 épocas y validación en un subconjunto de los datos.


In [23]:
early_stop = EarlyStopping(monitor="val_loss", patience=25, restore_best_weights=True) # Aumentado patience
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=7, min_lr=0.000005) # Aumentado patience y min_lr
filepath = "best_model.keras"
checkpoint = ModelCheckpoint(filepath, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max")

# Generación de texto en cada época
def generate_text(seed_text, next_words, model, tokenizer, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding="pre")[0]

        # Asegurarse de que token_list tenga la forma correcta para el modelo
        token_list = np.array(token_list).reshape(1, -1)

        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_word_index = np.argmax(predicted_probs)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

class TextGenerator(LambdaCallback):
    def __init__(self, seed_text, next_words, tokenizer, max_sequence_len):
        super().__init__()
        self.seed_text = seed_text
        self.next_words = next_words
        self.tokenizer = tokenizer
        self.max_sequence_len = max_sequence_len

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 == 0:
            generated = generate_text(self.seed_text, self.next_words, self.model, self.tokenizer, self.max_sequence_len)
            print(f"Epoch {epoch+1} - Generado: {generated}")

seed_text = "don quijote de la mancha"
text_generator_callback = TextGenerator(seed_text, 20, tokenizer, MAX_SEQUENCE_LEN)


history = model.fit(data_generator(sentences, tokenizer, MAX_SEQUENCE_LEN, total_words, BATCH_SIZE),
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=50,
                    verbose=1,
                    callbacks=[early_stop, reduce_lr, checkpoint, text_generator_callback],
                    validation_data=data_generator(sentences, tokenizer, MAX_SEQUENCE_LEN, total_words, BATCH_SIZE),
                    validation_steps=STEPS_PER_EPOCH // 5
                   )


Epoch 1/50
2879/2879 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.0667 - loss: 6.7566
Epoch 1: val_accuracy improved from -inf to 0.10247, saving model to best_model.keras
Epoch 1 - Generado: don quijote de la mancha y no no no no no no no no no no no no no no no no no no no
2879/2879 ━━━━━━━━━━━━━━━━━━━━ 111s 38ms/step - accuracy: 0.0667 - loss: 6.7565 - val_accuracy: 0.1025 - val_loss: 6.0207 - learning_rate: 0.0010
Epoch 2/50
2878/2879 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1100 - loss: 5.8717
Epoch 2: val_accuracy improved from 0.10247 to 0.12265, saving model to best_model.keras
2879/2879 ━━━━━━━━━━━━━━━━━━━━ 107s 37ms/step - accuracy: 0.1100 - loss: 5.8717 - val_accuracy: 0.1226 - val_loss: 5.6885 - learning_rate: 0.0010
Epoch 3/50
2879/2879 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1333 - loss: 5.5010
Epoch 3: val_accuracy improved from 0.12265 to 0.15417, saving model to best_model.keras
2879/2879 ━━━━━━━━━━━━━━━━━━━━ 143s 50ms/step - accuracy: 0.1333 - loss: 

In [24]:
def predict_next_word(seed_word, model, tokenizer, max_sequence_len):
    # Limpiar y tokenizar la palabra de inicio
    cleaned_word = unidecode(seed_word.lower())
    cleaned_word = re.sub(r'[^a-z¿¡.,;:!? ]', ' ', cleaned_word).strip()

    token_list = tokenizer.texts_to_sequences([cleaned_word])[0]

    if not token_list:
        return "Palabra no encontrada en el vocabulario."


    padded_sequence = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')[0]

    padded_sequence = np.array(padded_sequence).reshape(1, -1)

    predicted_probs = model.predict(padded_sequence, verbose=0)[0]
    predicted_word_index = np.argmax(predicted_probs)

    output_word = tokenizer.index_word.get(predicted_word_index, "<OOV>")

    return output_word

In [35]:
seed_text = "Don"
next_words_to_generate = 1 # Puedes ajustar este número
generated_sentence = generate_text(seed_text, next_words_to_generate, model, tokenizer, MAX_SEQUENCE_LEN)
print(generated_sentence)

Don quijote


# RNN Letra

##Tokenizar por Letra

Se crea un tokenizador a nivel de carácter (`char_level=True`) con un token especial `<OOV>` para caracteres desconocidos. Luego se ajusta al texto completo (`fit_on_texts`) para construir el índice de caracteres. Se calcula el total de caracteres únicos (`char_total`) y se convierte el texto en una secuencia de índices (`char_tokens`). Esto permite entrenar un modelo que predice el siguiente carácter en lugar de la siguiente palabra.

In [30]:
char_tokenizer = Tokenizer(char_level=True, oov_token='<OOV>')
char_tokenizer.fit_on_texts([text])

char_total = len(char_tokenizer.word_index) + 1
char_tokens = char_tokenizer.texts_to_sequences([text])[0]

print("Total de caracteres únicos:", char_total)

Total de caracteres únicos: 34


##Generación de sequencia de entrenamientos

Se define una longitud fija de secuencia (`seq_length = 40`) y se recorren los caracteres tokenizados para crear secuencias de entrada y salida. Cada secuencia contiene 40 caracteres como entrada (`X_char`) y el carácter siguiente como etiqueta (`y_char`). Se almacenan todas las secuencias como arrays de NumPy listos para entrenar el modelo. Esta técnica permite al modelo aprender patrones de caracteres para predecir el siguiente carácter en una secuencia dada.

In [31]:
seq_length = 40

input_sequences_char = []

for i in range(seq_length, len(char_tokens)):
    seq = char_tokens[i - seq_length:i + 1]
    input_sequences_char.append(seq)

# Convertir a array y separar en X e y
input_sequences_char = np.array(input_sequences_char)
X_char = input_sequences_char[:, :-1]
y_char = input_sequences_char[:, -1]

print("X_char shape:", X_char.shape)
print("y_char shape:", y_char.shape)


X_char shape: (2056136, 40)
y_char shape: (2056136,)


##Callbacks

###EarlyStop y reduce learning

In [36]:
early_stop_char = EarlyStopping(monitor='loss', patience=2, restore_best_weights=True)
reduce_lr_char = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=1)

###Texto Por epoca

Genera texto carácter a carácter al final de cada época, usando una semilla fija y muestreo con temperatura para predecir el siguiente carácter. Permite evaluar visualmente el avance del modelo durante el entrenamiento.

In [37]:
def on_epoch_end_generate_char(epoch, logs):
    print(f"\n Generación de texto (carácter a carácter) - Época {epoch + 1}")
    seed_text = "en un lugar de la mancha"
    result = seed_text
    for _ in range(300):
        token_list = char_tokenizer.texts_to_sequences([result[-40:]])[0]
        token_list = pad_sequences([token_list], maxlen=40, padding='pre')
        preds = model_char.predict(token_list, verbose=0)[0]

        preds = np.log(preds + 1e-8) / 0.8
        preds = np.exp(preds) / np.sum(np.exp(preds))
        next_index = np.random.choice(range(char_total), p=preds)
        next_char = char_tokenizer.index_word.get(next_index, '')

        result += next_char
    print("📝 Texto generado:", result)

generate_callback_char = LambdaCallback(on_epoch_end=on_epoch_end_generate_char)

###Gestion de modelos por epoca

Crea una carpeta específica para almacenar modelos carácter a carácter y define una ruta fija para sobrescribir el modelo después de cada época. El callback `save_callback_char_overwrite` guarda el modelo actual al finalizar cada época, asegurando que siempre exista una versión actualizada disponible.








In [38]:
# Carpeta específica para modelos por carácter
model_char_dir = "/content/drive/MyDrive/EV3/Checkpoints_char"
os.makedirs(model_char_dir, exist_ok=True)

previous_char_model_path = {"path": None}

In [39]:
# Ruta fija para guardar siempre el modelo por letra en el mismo archivo
overwrite_char_model_path = os.path.join(model_char_dir, "modelo_char_actual.keras")

def on_epoch_end_overwrite_char(epoch, logs):
    model_char.save(overwrite_char_model_path)
    print(f"💾 Modelo por letra sobrescrito en: {overwrite_char_model_path}")

save_callback_char_overwrite = LambdaCallback(on_epoch_end=on_epoch_end_overwrite_char)



##Modelo LSTM y Entrenamiento

In [40]:
# Definir modelo para carácter
model_char = Sequential()
model_char.add(Embedding(input_dim=char_total, output_dim=64))  # sin input_length
model_char.add(LSTM(128))
model_char.add(Dense(char_total, activation='softmax'))

# Construir con el largo correcto (seq_length = 40)
model_char.build(input_shape=(None, seq_length))

# Compilar
model_char.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Mostrar resumen
model_char.summary()



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 40, 64)         │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 34)             │         4,386 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 105,378 (411.63 KB)

 Trainable params: 105,378 (411.63 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_char.fit( X_char, y_char, epochs=30, batch_size=128, validation_split=0.1,
               callbacks=[ early_stop_char, reduce_lr_char, generate_callback_char, save_callback_char_overwrite   ])

Epoch 1/30
14455/14458 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4234 - loss: 1.8389
 Generación de texto (carácter a carácter) - Época 1
📝 Texto generado: en un lugar de la mancha la alguna alguna agrama se ardado. el entendre consigo don quijote y no has descaido, la de su concos oyendo aqui esta dando o de la honra quien esto, encantares y encigno que le proponido el albordamente, llenas dien tan moligase en mi tiendo temillantes antes. con tan enciempo anostemanza haber d
💾 Modelo por letra sobrescrito en: /content/drive/MyDrive/EV3/Checkpoints_char/modelo_char_actual.keras
14458/14458 ━━━━━━━━━━━━━━━━━━━━ 118s 8ms/step - accuracy: 0.4234 - loss: 1.8389 - val_accuracy: 0.5359 - val_loss: 1.4678 - learning_rate: 0.0010
Epoch 2/30
14451/14458 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5455 - loss: 1.4254
 Generación de texto (carácter a carácter) - Época 2
📝 Texto generado: en un lugar de la mancha que, a surenta con compantado a mucha, sancho panza , y con el, parrona segur

Carga un modelo previamente entrenado y, dado un texto semilla, predice el siguiente carácter usando muestreo con temperatura. Convierte la semilla en secuencia, la ajusta al tamaño requerido (`seq_length`) y devuelve el carácter más probable según la distribución generada.


In [41]:
def predict_next_char_from_path(seed_text, path_modelo, tokenizer, seq_length, vocab_size, temperature=1.0):
    model = load_model(path_modelo)

    seed_text = seed_text.lower()
    token_list = tokenizer.texts_to_sequences([seed_text[-seq_length:]])[0]
    token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')

    preds = model.predict(token_list, verbose=0)[0]
    preds = np.log(preds + 1e-8) / temperature
    preds = np.exp(preds) / np.sum(np.exp(preds))

    next_index = np.random.choice(range(vocab_size), p=preds)
    next_char = tokenizer.index_word.get(next_index, '')
    return next_char

In [43]:
ruta_modelo = "/content/drive/MyDrive/EV3/Checkpoints_char/modelo_char_actual.keras"
predict_next_char_from_path("sanch", ruta_modelo, char_tokenizer, seq_length=40, vocab_size=char_total)

'o'